# Test reading a set of rainfields netCDF rain_rate files from the mongo database

In [1]:
import datetime
import os
import gridfs as fs
from pymongo import MongoClient
from netCDF4 import Dataset
from package import rain_base_fs


In [2]:
def get_files(**kwargs):
    """ Return a list of files that match the search criteria or None on error
    """
    config = kwargs.get("config", None)

    # get configuration paramaters
    start_time = datetime.datetime.fromisoformat(
        config["START_DATE"]).replace(tzinfo=datetime.timezone.utc)
    end_time = datetime.datetime.fromisoformat(
        config["END_DATE"]).replace(tzinfo=datetime.timezone.utc)
    station_id = int(config["STN_ID"])
    product = config["PRODUCT"]

    query = {}
    query['metadata.valid_time'] = {'$gte': int(datetime.datetime.timestamp(
        start_time)), '$lte': int(datetime.datetime.timestamp(end_time))}
    query['metadata.station_id'] = station_id
    query['metadata.product'] = product

    client = MongoClient()
    db = client[config["DB_NAME"]]
    cursor = db.fs.files.find(query).sort([("metadata.valid_time",1)]) 

    file_list = []
    if cursor is None:
        print("None found") 
        return None
    else:
        for doc in cursor:
            file_list.append(doc["filename"])
    
    client.close()
    return file_list


In [3]:
config = {
    "START_DATE": "2021-10-04T04:15:00",
    "END_DATE": "2021-10-05T01:00:00",
    "PRODUCT": "prcp-r15",
    "STN_ID": "76",
    "DB_NAME": "radar",
    "VARIABLE": "precipitation"
}


In [4]:
# get the list of file names that match this search 
file_list = get_files(config=config)
print(file_list)

['76_20211004_041500.prcp-r15.nc', '76_20211004_043000.prcp-r15.nc', '76_20211004_044500.prcp-r15.nc', '76_20211004_050000.prcp-r15.nc', '76_20211004_051500.prcp-r15.nc', '76_20211004_053000.prcp-r15.nc', '76_20211004_054500.prcp-r15.nc', '76_20211004_060000.prcp-r15.nc', '76_20211004_061500.prcp-r15.nc', '76_20211004_063000.prcp-r15.nc', '76_20211004_064500.prcp-r15.nc', '76_20211004_070000.prcp-r15.nc', '76_20211004_071500.prcp-r15.nc', '76_20211004_073000.prcp-r15.nc', '76_20211004_074500.prcp-r15.nc', '76_20211004_080000.prcp-r15.nc', '76_20211004_081500.prcp-r15.nc', '76_20211004_083000.prcp-r15.nc', '76_20211004_084500.prcp-r15.nc', '76_20211004_090000.prcp-r15.nc', '76_20211004_091500.prcp-r15.nc', '76_20211004_093000.prcp-r15.nc', '76_20211004_094500.prcp-r15.nc', '76_20211004_100000.prcp-r15.nc', '76_20211004_101500.prcp-r15.nc', '76_20211004_103000.prcp-r15.nc', '76_20211004_104500.prcp-r15.nc', '76_20211004_110000.prcp-r15.nc', '76_20211004_111500.prcp-r15.nc', '76_20211004_

In [5]:
# loop over the filenames and read in the data 
client = MongoClient()
db = client[config["DB_NAME"]]
radar_fs = fs.GridFS(db)
for filename in file_list:
    file = radar_fs.find_one({"filename":filename})
    data = file.read()
    
    ncFile = Dataset(filename,mode="r",memory=data)
    station_id = int(ncFile.__getattr__("station_id"))
    station_name = str(ncFile.__getattr__("station_name"))
    vtime = int(ncFile['valid_time'][0].item())
    valid_time = datetime.datetime.fromtimestamp(vtime,tz=datetime.timezone.utc)
    print(f"{station_id=}, {station_name=},{valid_time.isoformat()}")
    ncFile.close()

client.close()


station_id=76, station_name='Koonya',2021-10-04T04:15:00+00:00
station_id=76, station_name='Koonya',2021-10-04T04:30:00+00:00
station_id=76, station_name='Koonya',2021-10-04T04:45:00+00:00
station_id=76, station_name='Koonya',2021-10-04T05:00:00+00:00
station_id=76, station_name='Koonya',2021-10-04T05:15:00+00:00
station_id=76, station_name='Koonya',2021-10-04T05:30:00+00:00
station_id=76, station_name='Koonya',2021-10-04T05:45:00+00:00
station_id=76, station_name='Koonya',2021-10-04T06:00:00+00:00
station_id=76, station_name='Koonya',2021-10-04T06:15:00+00:00
station_id=76, station_name='Koonya',2021-10-04T06:30:00+00:00
station_id=76, station_name='Koonya',2021-10-04T06:45:00+00:00
station_id=76, station_name='Koonya',2021-10-04T07:00:00+00:00
station_id=76, station_name='Koonya',2021-10-04T07:15:00+00:00
station_id=76, station_name='Koonya',2021-10-04T07:30:00+00:00
station_id=76, station_name='Koonya',2021-10-04T07:45:00+00:00
station_id=76, station_name='Koonya',2021-10-04T08:00:0